In [ ]:
using BayesNets

In [ ]:
f = Factor([:a, :b], [2, 2])
g = Factor([:b, :c], [2, 2])
factors = [f, g]
mrf = MRF(factors)

In [ ]:
names(mrf)

In [ ]:
mrf.ug

In [ ]:
fieldnames(mrf.ug)

In [ ]:
mrf.ug.vertices

In [ ]:
neighbors(mrf,NodeName(:b))

In [ ]:
get(mrf, 1)

In [ ]:
markov_blanket(mrf, NodeName(:a))

In [ ]:
has_edge(mrf, NodeName(:a), NodeName(:b))

In [ ]:
g2 = copy(mrf.ug)

In [ ]:
mrf.name_to_index[:a]

In [ ]:
neighbors(g2, 2)

In [ ]:
using LightGraphs
rem_edge!(g2, 2, 3)

In [ ]:
connected_components(g2)

In [ ]:
array1 = [1, 2 ,3 ,4]

In [ ]:
array2 = [4, 5, 1 , 919]

In [ ]:
intersect(array1, array2)

In [ ]:
if intersect(array1, array2)
    print("hi")
end

In [ ]:
not isempty(array1

In [ ]:
not(isempty([]))

In [ ]:
f = Factor([:a, :b], [2, 2])
g = Factor([:b, :c], [2, 2])
h = Factor([:c, :f], [2, 2])
i = Factor([:a, :d], [2, 2])
j = Factor([:d, :e], [2, 2])
k = Factor([:d, :g], [2, 2])
l = Factor([:b, :e], [2, 2])
m = Factor([:f, :e], [2, 2])
n = Factor([:h, :e], [2, 2])
o = Factor([:g, :h], [2, 2])
p = Factor([:i, :h], [2, 2])
q = Factor([:i, :f], [2, 2])


factors = [f, g, h, i, j, k,l, m,n, o, p, q]
mrf = MRF(factors)

In [ ]:
using Plots
using PlotRecipes

In [6]:
Pkg.add("GR")

INFO: Cloning cache of GR from https://github.com/jheinen/GR.jl.git
INFO: Installing GR v0.19.0
INFO: Building GR
INFO: Downloading pre-compiled GR 0.23.0 Ubuntu binary
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 38.1M  100 38.1M    0     0  2551k      0  0:00:15  0:00:15 --:--:-- 5838k
INFO: Package database updated


In [7]:
using BayesNets
using LightGraphs
using Plots
gr(size = (200,200), scale = 1)

INFO: Precompiling module GR.


Plots.GRBackend()

In [20]:
size = 20
grid = Grid([size,size])

{400, 760} undirected graph

In [21]:
interact = .8
interact_array = [exp(interact) exp(-interact) ; exp(-interact) exp(interact)]

factors = Array{Factor}(length(edges(grid)))

for (i,e) in enumerate(edges(grid))
    factors[i] = Factor([Symbol(e[1]), Symbol(e[2])], interact_array)
end

mrf = MRF(factors);

In [22]:
sampler = MRFGibbsSampler(Assignment(), burn_in = 0)

BayesNets.MRFGibbsSampler(Dict{Symbol,Any}(),0,0,Nullable{Dict{Symbol,Any}}())

In [23]:
iters = 50
@time df = rand(mrf, sampler, iters);

325.932719 seconds (1.66 G allocations: 158.937 GB, 7.29% gc time)


In [24]:
a = reshape(convert(Array{Float64},df[1,:]), (size, size) )
plt = heatmap(a, axis = nothing)

@gif for i=1:iters
    a = reshape(convert(Array{Float64}, df[i,:]), (size, size))
    heatmap!(a, axis = nothing)
end

INFO: Saved animation to /home/henripal/.julia/v0.5/BayesNets.jl/doc/tmp.gif


Plots.AnimatedGif("/home/henripal/.julia/v0.5/BayesNets.jl/doc/tmp.gif")